In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, f1_score
from keras import layers, models, optimizers
from keras.applications import VGG19

In [22]:
# Function to load and preprocess images
def load_and_preprocess_images(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            if image is not None:
                # Resize the image to the target size (adjust as needed)
                image = cv2.resize(image, (224, 224))
                images.append(image)
                labels.append(label)
    return images, labels


In [23]:
# Path to the "train" directory
dataset_dir = "train"

In [24]:


# Load and preprocess images from "Open" and "Closed" folders
open_folder = os.path.join(dataset_dir, "Open")
closed_folder = os.path.join(dataset_dir, "Closed")

if not os.path.exists(open_folder) or not os.path.exists(closed_folder):
    print("Error: 'Open' or 'Closed' folders not found.")
    exit()

open_eye_data, open_eye_labels = load_and_preprocess_images(open_folder, label=1)
closed_eye_data, closed_eye_labels = load_and_preprocess_images(closed_folder, label=0)

# Combine data and labels
data = np.vstack((open_eye_data, closed_eye_data))
labels = np.concatenate((open_eye_labels, closed_eye_labels))

# Print debug information
print("Number of samples in 'Open' class:", len(open_eye_data))
print("Number of samples in 'Closed' class:", len(closed_eye_data))
print("Total number of samples:", len(data))

# Split the data into training and testing sets
if len(data) >= 2:
    train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)
else:
    print("Error: Insufficient data for splitting. Ensure you have at least 2 samples.")
    # Handle the error or adjust the dataset accordingly
    exit()

# Preprocess the data for VGG-19
if len(train_data) >= 1:
    train_data = preprocess_input(train_data)
    test_data = preprocess_input(test_data)
else:
    print("Error: Insufficient data for preprocessing. Ensure you have at least 1 sample.")
    # Handle the error or adjust the dataset accordingly
    exit()

Number of samples in 'Open' class: 726
Number of samples in 'Closed' class: 726
Total number of samples: 1452


In [25]:
# Build the VGG-19 model
vgg_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

model = models.Sequential()
model.add(vgg_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
batch_size = 32
history = model.fit(train_data, train_labels, epochs=epochs, batch_size=batch_size, validation_split=0.2)

# Evaluate the model on the testing data
test_predictions = model.predict(test_data)
test_predictions_binary = (test_predictions > 0.5).astype(int)

# Calculate metrics
test_accuracy = accuracy_score(test_labels, test_predictions_binary)
test_precision = precision_score(test_labels, test_predictions_binary)
test_f1 = f1_score(test_labels, test_predictions_binary)

print(f'Test Accuracy: {test_accuracy:.4f}')
print(f'Test Precision: {test_precision:.4f}')
print(f'Test F1 Score: {test_f1:.4f}')

Epoch 1/10
29/29 [==============================] - 1203s 41s/step - loss: 0.7654 - accuracy: 0.9235 - val_loss: 0.0611 - val_accuracy: 0.9785
Epoch 2/10
29/29 [==============================] - 1203s 42s/step - loss: 0.0313 - accuracy: 0.9892 - val_loss: 0.0296 - val_accuracy: 0.9957
Epoch 3/10
17/29 [================>.............] - ETA: 7:49 - loss: 0.0349 - accuracy: 0.9908